# IMPORT LIBRARIES AND DATABASE AND COLLECTIONS

In [1]:
import pymongo
from datetime import datetime
import uuid

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access the database
db = client['yelp']  

# Access the collections
review_collection = db['review']
user_collection = db['user']
business_collection = db['business']
admin_collection = db['admin']

# AUFGABE 1
### Möglichkeit 1

Search for reviews of a business

In [2]:
def authenticate_user(admin_username, admin_password):
    existing_admin = admin_collection.find_one({"admin_username": admin_username})

    if existing_admin is None:
        return None, "Wrong username. Please enter the right username and password."

    # Check if the password matches
    if existing_admin['admin_password'] != admin_password:
        return None, "Wrong password. Please enter the right username and password."

    business_id = existing_admin['business_id']

    # Retrieve reviews for the specified business_id
    reviews = review_collection.find({"business_id": business_id})
    return list(reviews)

# Beispielaufruf:
admin_username = input("Admin Username: ")
admin_password = input("Admin Password: ")

results = authenticate_user(admin_username, admin_password)
for result in results:
    print(result)

{'_id': ObjectId('656b238d8895f3b51c4d3e57'), 'review_id': '9vwYDBVI3ymdqcyJ5WW2Tg', 'user_id': 'e0imecnX_9MtLnS2rUZM-A', 'business_id': 'Pns2l4eNsfO8kk83dixA6A', 'stars': 5, 'useful': 3, 'funny': 2, 'cool': 1, 'text': "I've had acupuncture treatments with Abby over the past few years and I really can't say enough about how great my experiences have been.\n\nShe has successfully treated me for everything from stress to colds to menstrual cramps.  I was amazed that her medical qi gong treatments helped alleviate my chronic back pain.  I didn't think anything could help with that!  \n\nShe is always very clear and upfront about treatments and expectations.   And I love that she always has great advice on herbs, supplements and natural remedies.\n\nShe just relocated to Los Gatos and I'm definitely going to miss having her in San Francisco!", 'date': '2012-05-02 18:07:38'}
{'_id': ObjectId('656b23928895f3b51c4e06d9'), 'review_id': 'OXgg1LdxHDv3CBU5-xi2lA', 'user_id': '_Q0fdLVoTnlNkEypUvNk

### Möglichkeit 2

filter timerange and stars

In [3]:
def authenticate_user(admin_username, admin_password, start_date, end_date, min_stars, max_stars):
    existing_admin = admin_collection.find_one({"admin_username": admin_username})

    if existing_admin is None:
        return None, "Wrong username. Please enter the right username and password."

    # Check if the password matches
    if existing_admin['admin_password'] != admin_password:
        return None, "Wrong password. Please enter the right username and password."

    business_id = existing_admin['business_id']

    # Format start_date and end_date into strings matching the database format
    formatted_start_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    formatted_end_date = end_date.strftime("%Y-%m-%d %H:%M:%S")

    # Retrieve reviews for the specified business_id within the time range and star range
    reviews = review_collection.find({
        "business_id": business_id,
        "date": {"$gte": formatted_start_date, "$lte": formatted_end_date},
        "stars": {"$gte": min_stars, "$lte": max_stars}
    })
    return list(reviews), None

# Beispielaufruf:
admin_username = input("Admin Username: ")
admin_password = input("Admin Password: ")

start_year = int(input("Start Year: "))
start_month = int(input("Start Month: "))
start_day = int(input("Start Day: "))
start_hour = int(input("Start Hour: "))
start_minute = int(input("Start Minute: "))
start_second = int(input("Start Second: "))

end_year = int(input("End Year: "))
end_month = int(input("End Month: "))
end_day = int(input("End Day: "))
end_hour = int(input("End Hour: "))
end_minute = int(input("End Minute: "))
end_second = int(input("End Second: "))

# Define the start and end dates for the time range (modify as needed)
start_date = datetime(start_year, start_month, start_day, start_hour, start_minute, start_second)
end_date = datetime(end_year, end_month, end_day, end_hour, end_minute, end_second)

# Define the star rating range (modify as needed)
min_stars = 3
max_stars = 5

results, error = authenticate_user(admin_username, admin_password, start_date, end_date, min_stars, max_stars)
if error:
    print(error)
else:
    for result in results:
        print(result)

{'_id': ObjectId('656b238d8895f3b51c4d3e57'), 'review_id': '9vwYDBVI3ymdqcyJ5WW2Tg', 'user_id': 'e0imecnX_9MtLnS2rUZM-A', 'business_id': 'Pns2l4eNsfO8kk83dixA6A', 'stars': 5, 'useful': 3, 'funny': 2, 'cool': 1, 'text': "I've had acupuncture treatments with Abby over the past few years and I really can't say enough about how great my experiences have been.\n\nShe has successfully treated me for everything from stress to colds to menstrual cramps.  I was amazed that her medical qi gong treatments helped alleviate my chronic back pain.  I didn't think anything could help with that!  \n\nShe is always very clear and upfront about treatments and expectations.   And I love that she always has great advice on herbs, supplements and natural remedies.\n\nShe just relocated to Los Gatos and I'm definitely going to miss having her in San Francisco!", 'date': '2012-05-02 18:07:38'}
{'_id': ObjectId('656b23928895f3b51c4e06d9'), 'review_id': 'OXgg1LdxHDv3CBU5-xi2lA', 'user_id': '_Q0fdLVoTnlNkEypUvNk

# AUFGABE 2
### INSERT ADMIN DATA

In [4]:
def username_exist(username):
    # Check if the username exists in the admin collection
    existing_admin = admin_collection.find_one({"admin_username": username})
    return existing_admin is not None

def add_admin(business_name, business_address, business_city, admin_name, admin_username, 
              admin_email, admin_password, admin_password_confirm, threshold_percentage, last_n_reviews):
    if username_exist(admin_username):
        return "Username already exists for another admin"
    
    business = business_collection.find_one({"name": business_name, "address": business_address, "city": business_city})
    if not business:
        return "Business not found"
    business_id = business['business_id']

    existing_admin_with_business = admin_collection.find_one({"business_id": business_id})
    if existing_admin_with_business:
        return "Business already exists for another admin"

    if admin_password != admin_password_confirm:
        return "Passwords do not match"
    
    admin_id = str(uuid.uuid4())

    # Construct the admin document
    admin_doc = {
        "_id": admin_id,
        "admin_id": str(uuid.uuid4()),
        "admin_name": admin_name,
        "business_id": business_id,
        "admin_username": admin_username,
        "admin_email": admin_email,
        "admin_password": admin_password,
        "threshold_percentage": int(threshold_percentage),
        "last_n_reviews": int(last_n_reviews)
    }

    # Insert the admin document into the admin collection
    admin_collection.insert_one(admin_doc)
    return "Admin data added successfully"

# Example usage:
result = add_admin(
    business_name=input("Business Name: "), 
    business_address=input("Business Address: "),
    business_city=input("Business City: "),
    admin_name=input("Admin Name: "), 
    admin_username=input("Admin Username: "), 
    admin_email=input("Email: "), 
    admin_password=input("Password: "), 
    admin_password_confirm=input("Confirm Password: "), 
    threshold_percentage=input("Threshold percentage: "), 
    last_n_reviews=input("Last N Reviews: ")
    )
print(result)

Business not found


### INSERT REVIEW DATA

In [5]:
def add_review(user_name, business_name, business_address, 
               business_city, stars, useful, funny, cool, text, ):
    # Generate new random IDs for review and user
    review_id = str(uuid.uuid4())
    user_id = str(uuid.uuid4())

    # Get current date and time
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Find business_id based on business_name, business_address, and business_city
    business = business_collection.find_one({"name": business_name, "address": business_address, "city": business_city})
    if not business:
        return "Business not found"
    business_id = business['business_id']

    # Find user_id based on user_name
    user = user_collection.find_one({"name": user_name})
    if not user:
        return "User not found"
    user_id = user['user_id']

    stars = int(stars)
    useful = int(useful)
    funny = int(funny)
    cool = int(cool)

    # Construct the review document
    review_doc = {
        "_id": review_id,
        "review_id": str(uuid.uuid4()),
        "user_id": user_id,
        "business_id": business_id,
        "stars": stars,
        "useful": useful,
        "funny": funny,
        "cool": cool,
        "text": text,
        "date": current_date
    }

    # Insert the review document into the review collection
    review_collection.insert_one(review_doc)
    return "Review added successfully"

# Example usage:
result = add_review(
    user_name=input("User Name: "),
    business_name=input("Business Name: "),
    business_address=input("Business Address: "),
    business_city=input("Business City: "),
    stars=input("How many stars do you want to give? "),
    useful=input("How useful are they to you? "),
    funny=input("How funny are they? "),
    cool=input("How cool are they? "),
    text=input("Comment: ")
    )
print(result)

Business not found
